In [1]:
import pandas as pd
import numpy as np
import geemap
import folium
import streamlit as st
import geopandas as gpd
from pathlib import Path
from shapely.geometry import Point
from colour import Color





# LM sampling


- Set up paths


In [2]:
PROJECT_ROOT = Path().absolute().parent
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'



## radio station locations




In [3]:
station_loc_df = pd.read_csv(RAW_DATA_DIR/"station_loc.csv", sep=';')


In [4]:
station_loc_df
#convert to geodataframe. 
station_loc_gdf = gpd.GeoDataFrame(
    station_loc_df, 
    geometry=[Point(xy) for xy in zip(station_loc_df['longitude'], station_loc_df['latitude'])],  # Note: longitude first, then latitude
    crs="EPSG:4326"
)
colors = ['#1b9e77', '#d95f02', '#7570b3']
#adding colors to gdf
station_loc_gdf['color']=colors
station_loc_gdf
station_loc_gdf.to_file(PROCESSED_DATA_DIR / 'station_loc.gpkg')



In [5]:
# create a buffer around each radio station. 

def create_station_buffers(gdf, buffer_distances=[20000, 25000, 40000, 60000]):
    """
    Create buffer rings around radio stations with lighter colors for larger buffers
    """
    # Convert to UTM for accurate distances
    gdf_proj = gdf.to_crs('EPSG:32636')
    
    buffer_list = []
    
    # Create color gradients for each station
    for idx, row in gdf_proj.iterrows():
        base_color = Color(row['color'])
        # Create lighter versions - more distances = more colors needed
        colors = list(base_color.range_to(Color('white'), len(buffer_distances) + 1))[:-1]
        
        for distance, color in zip(buffer_distances, colors):
            buffer_list.append({
                'station_name': row['station_name'],
                'buffer_km': distance/1000,
                'original_color': row['color'],
                'buffer_color': color.hex,  # lighter version of original color
                'geometry': row.geometry.buffer(distance)
            })
    
    buffers_gdf = gpd.GeoDataFrame(buffer_list, crs=gdf_proj.crs)
    return buffers_gdf.to_crs('EPSG:4326')



station_buffers_gdf=create_station_buffers(station_loc_gdf, buffer_distances=[20000, 25000, 40000, 60000])




In [6]:
#exporting station buffers
station_buffers_gdf.to_file(PROCESSED_DATA_DIR / 'station_buffers.gpkg')


In [7]:
# Create map
m = folium.Map(location=[station_loc_gdf.geometry.y.mean(), station_loc_gdf.geometry.x.mean()], 
               zoom_start=10)

# Create legend
legend_html = """
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 200px; height: auto;
            background-color: white;
            border: 2px solid grey;
            z-index: 1000;
            padding: 10px;
            font-size: 14px;
            ">
<p><strong>Radio Stations</strong></p>
"""

# Add each station to the legend with its color
for idx, row in station_loc_gdf.iterrows():
    legend_html += f"""
    <p>
        <i class="fa fa-circle" style="color:{row['color']}"></i>
        {row['station_name']}
    </p>
    """
legend_html += "<p><strong>Assumed Coverage Ranges</strong></p>"
legend_html += """
<p>― 20km (solid line, highest opacity)</p>
<p>-- 25km (dashed line)</p>
<p>― ― 40km (long dashes)</p>
<p>... 60km (dotted line, lowest opacity)</p>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

# Create a Feature Group for each radio station's buffers
station_buffer_groups = {}
for station in station_loc_gdf['station_name'].unique():
    station_buffer_groups[station] = folium.FeatureGroup(name=f'Ranges {station}')

# Sort buffers from largest to smallest for proper layering
station_buffers_gdf = station_buffers_gdf.sort_values('buffer_km', ascending=False)

# Add buffers with different styles
for idx, row in station_buffers_gdf.iterrows():
    station_name = row['station_name']
    
    # Define style based on buffer size
    if row['buffer_km'] == 20:
        style = {
            'fillOpacity': 0.4, 
            'dashArray': None, 
            'weight': 2
        }
    elif row['buffer_km'] == 25:
        style = {
            'fillOpacity': 0.3, 
            'dashArray': '5,5', 
            'weight': 2
        }
    elif row['buffer_km'] == 40:
        style = {
            'fillOpacity': 0.2, 
            'dashArray': '10,10', 
            'weight': 2
        }
    else:  # 60km
        style = {
            'fillOpacity': 0.1, 
            'dashArray': '2,8', 
            'weight': 2
        }

    folium.GeoJson(
        row.geometry,
        style_function=lambda x, color=row['original_color'], style=style: {
            'fillColor': color,
            'color': color,
            'fillOpacity': style['fillOpacity'],
            'dashArray': style['dashArray'],
            'weight': style['weight']
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': style['fillOpacity'] + 0.2
        },
        tooltip=f"{station_name} - {row['buffer_km']}km range",
        popup=folium.Popup(
            f"""
            <div style='width: 200px'>
                <b>{station_name}</b><br>
                Coverage Range: {row['buffer_km']} km<br>
            </div>
            """,
            max_width=300
        )
    ).add_to(station_buffer_groups[station_name])

# Add station points
for idx, row in station_loc_gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['station_name'],
        tooltip=f"Click to see {row['station_name']} location",
        icon=folium.Icon(
            color='white',  
            icon_color=row['color'],
            icon='radio', 
            prefix='fa'
        )
    ).add_to(m)

# Add all buffer groups to the map
for group in station_buffer_groups.values():
    group.add_to(m)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

m